In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler,MinMaxScaler,MaxAbsScaler
from sklearn.metrics import accuracy_score,auc,recall_score,precision_score,precision_recall_curve,confusion_matrix
from xgboost import XGBClassifier
#pd.set_option("display.max_columns",50)

In [3]:
path = "C:/Users/Krishna/Documents/kaggle/titanic/input"

##### Reading train csv

In [4]:
train = pd.read_csv(f'{path}/train.csv')
train.head(2)
train.shape

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


(891, 12)

In [5]:
train['Pclass'] = train['Pclass'].astype(str)
train.describe()
train.info()

,PassengerId,Survived,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,80.000000,8.000000,6.000000,512.329200


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null object
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 83.7+ KB


In [6]:
#train.Name.map(re.search("(.*)\\, (.*)\\.(.*) ").groups())
train['Surname'] = train.Name.map(str.upper).map(lambda x : re.search("(.*)\\, (.*?)\\.(.*)",x).group(2))

In [7]:
a = train.Surname.value_counts(normalize = True)>0.04
a = a[a!=True].index

#train.Surname[train.Surname.isin(a)] = 'Others'
train.Surname = np.where(~train.Surname.isin(a),train.Surname,'Others')
train.Surname.value_counts()

#train.Surname[~train.Surname.isin(a)] = train.Surname[~train.Sir_name.isin(a)].map(dict(zip(a,['Others']*len(a))))

MR        517
MISS      182
MRS       125
MASTER     40
Others     27
Name: Surname, dtype: int64

In [8]:
train.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Surname
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,MR


In [9]:
train.Embarked = np.where(train.Embarked.isna(),train.Embarked.mode(dropna=True), train.Embarked)

train.Embarked.isna().sum()
train.Embarked.value_counts()

0

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [10]:
test = pd.read_csv(f'{path}/test.csv')
test.head(2)
test.shape

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S


(418, 11)

In [11]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [12]:
pd.crosstab(train['Surname'],train['Sex'])

Sex,female,male
Surname,,
MASTER,0,40
MISS,182,0
MR,0,517
MRS,125,0
Others,7,20


In [13]:
train.groupby('Surname').Age.mean()

Surname
MASTER     4.574167
MISS      21.773973
MR        32.368090
MRS       35.898148
Others    42.384615
Name: Age, dtype: float64

In [14]:
train.groupby('Surname').Age.mean()
#train.Age.dropna().groupby(train.Surname, sort = False).mean()
train.Age.isna().groupby(train.Surname, sort = False).sum()

Surname
MASTER     4.574167
MISS      21.773973
MR        32.368090
MRS       35.898148
Others    42.384615
Name: Age, dtype: float64

Surname
MR        119.0
MRS        17.0
MISS       36.0
MASTER      4.0
Others      1.0
Name: Age, dtype: float64

In [15]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [16]:
test['Surname'] = test.Name.map(str.upper).map(lambda x : re.search("(.*)\\, (.*?)\\.(.*)",x).group(2))
test.Surname = np.where(test.Surname.isin(['MR','MISS','MRS','MASTER']),test.Surname,'Others')


test.Surname.isin(a).value_counts()

False    418
Name: Surname, dtype: int64

In [17]:
test['Surname'].value_counts()

MR        240
MISS       78
MRS        72
MASTER     21
Others      7
Name: Surname, dtype: int64

In [18]:
test.groupby('Surname').Age.mean()
#test.Age.dropna().groupby(test.Surname, sort = False).mean()
test.Age.isna().groupby(test.Surname, sort = False).sum()

Surname
MASTER     7.406471
MISS      21.774844
MR        32.000000
MRS       38.903226
Others    43.833333
Name: Age, dtype: float64

Surname
MR        57.0
MRS       10.0
MISS      14.0
MASTER     4.0
Others     1.0
Name: Age, dtype: float64

In [19]:
train.loc[train.Age.isna(), 'Age'] = train[train.Age.isna()].Surname.map(train.groupby('Surname').Age.mean().to_dict())

In [20]:
test.Fare.isna().sum()
test['Fare'] = test.Fare.fillna(test.Fare.mean())
test.Fare.isna().sum()

1

0

In [21]:
test.loc[test.Age.isna(), 'Age'] = test[test.Age.isna()].Surname.map(train.groupby('Surname').Age.mean().to_dict())

In [22]:
train.isna().sum()
test.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
Surname          0
dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
Surname          0
dtype: int64

In [23]:
train[['Sex','Pclass', 'Embarked','Surname']] = train[['Sex','Pclass', 'Embarked','Surname']].astype(str)

test[['Sex','Pclass', 'Embarked','Surname']] = test[['Sex','Pclass', 'Embarked','Surname']].astype(str)

In [24]:
def dict_vec(train_set,cols,is_test, test_set):
    """
    returns dict vectorizer on train set or train & test set for chosen columns
    train_set: Dataset on which DV is to be fit
    cols: List of columns of train_set which are to be considered for DV
    is_test: Boolean, If DV is to be transformed on test too
    test_set: Test set on which DV is to be transformed
    """
    from sklearn.feature_extraction import DictVectorizer
    import pandas as pd
    dvec = DictVectorizer(sparse=False)
    if not is_test:
        test_set = pd.DataFrame()
        train_dvec = dvec.fit_transform(train_set[cols].transpose().to_dict().values())
        train_dvec = pd.DataFrame(train_dvec, index = train_set.index, columns = dvec.get_feature_names())
        train_df = pd.concat([train_set.drop(cols, axis = 1),train_dvec], axis = 1)
        return train_df,pd.DataFrame(),dvec
    else:
        train_dvec = dvec.fit_transform(train_set[cols].transpose().to_dict().values())
        train_dvec = pd.DataFrame(train_dvec, index = train_set.index, columns = dvec.get_feature_names())
        train_df = pd.concat([train_set.drop(cols, axis = 1),train_dvec], axis = 1)
        test_dvec = dvec.transform(test_set[cols].transpose().to_dict().values())
        test_dvec = pd.DataFrame(test_dvec, index = test_set.index, columns = dvec.get_feature_names())
        test_df = pd.concat([test_set.drop(cols, axis = 1),test_dvec], axis = 1)
        return train_df, test_df,dvec

In [25]:
train_df,test_df,dv = dict_vec(train, ['Sex','Pclass', 'Embarked','Surname'],is_test = True,test_set =test)

In [26]:
train_df.columns

Index(['PassengerId', 'Survived', 'Name', 'Age', 'SibSp', 'Parch', 'Ticket',
       'Fare', 'Cabin', 'Embarked=C', 'Embarked=Q', 'Embarked=S', 'Pclass=1',
       'Pclass=2', 'Pclass=3', 'Sex=female', 'Sex=male', 'Surname=MASTER',
       'Surname=MISS', 'Surname=MR', 'Surname=MRS', 'Surname=Others'],
      dtype='object')

In [27]:
target  = 'Survived'
features= train_df.drop(['PassengerId','Survived', 'Name','Ticket','Cabin'], axis =1).columns
target
features

'Survived'

Index(['Age', 'SibSp', 'Parch', 'Fare', 'Embarked=C', 'Embarked=Q',
       'Embarked=S', 'Pclass=1', 'Pclass=2', 'Pclass=3', 'Sex=female',
       'Sex=male', 'Surname=MASTER', 'Surname=MISS', 'Surname=MR',
       'Surname=MRS', 'Surname=Others'],
      dtype='object')

### splitting into train and test set

In [28]:

X_train,X_test,y_train,y_test = train_test_split(train_df[features],train_df[target],test_size=0.25, random_state=69)

##### fitting on train set

In [29]:

sc = MaxAbsScaler()
logreg = LogisticRegression(solver = 'lbfgs', max_iter=10000,C= 0.1)
#res = df.apply(lambda x: x.fillna(0) if x.dtype.kind in 'biufc' else x.fillna('.'))

In [30]:
logreg.fit(X_train[features], y_train)

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=10000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
pred_bin = logreg.predict(X_train[features])
pred_prob = logreg.predict_proba(X_train[features])[:,1]
logreg.score(X_train[features], y_train) # accuracy

0.8248502994011976

In [32]:
precision_score(y_train,pred_bin)
recall_score(y_train,pred_bin)
confusion_matrix(y_train,pred_bin)
accuracy_score(y_train,pred_bin)

0.8114035087719298

0.7142857142857143

array([[366,  43],
       [ 74, 185]], dtype=int64)

0.8248502994011976

### logreg on test

In [33]:
pred_bin = logreg.predict(X_test[features])
pred_prob = logreg.predict_proba(X_test[features])[:,1]
logreg.score(X_test[features], y_test) # accuracy

0.820627802690583

In [34]:
precision_score(y_test,pred_bin)
recall_score(y_test,pred_bin)
confusion_matrix(y_test,pred_bin)
accuracy_score(y_test,pred_bin)

0.7721518987341772

0.7349397590361446

array([[122,  18],
       [ 22,  61]], dtype=int64)

0.820627802690583

In [35]:

#XGB = XGBClassifier(n_estimators = 300,learning_rate= 0.01)
XGB = RandomForestClassifier(n_estimators=750,oob_score=True,max_depth=6)
XGB



RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=6, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=750,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [36]:
XGB.fit(X_train[features], y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=6, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=750,
                       n_jobs=None, oob_score=True, random_state=None,
                       verbose=0, warm_start=False)

In [37]:
pred_bin = XGB.predict(X_train)
pred_prob = XGB.predict_proba(X_train)[:,1]
XGB.score(X_train, y_train) # accuracy

0.8652694610778443

In [38]:
precision_score(y_train,pred_bin)
recall_score(y_train,pred_bin)
confusion_matrix(y_train,pred_bin)
accuracy_score(y_train,pred_bin)

0.9004739336492891

0.7335907335907336

array([[388,  21],
       [ 69, 190]], dtype=int64)

0.8652694610778443

##### prediction on test

In [39]:
pred_bin = XGB.predict(X_test)
pred_prob = XGB.predict_proba(X_test)[:,1]
XGB.score(X_test, y_test) # accuracy

0.8475336322869955

In [40]:
precision_score(y_test,pred_bin)
recall_score(y_test,pred_bin)
confusion_matrix(y_test,pred_bin)
accuracy_score(y_test,pred_bin)

0.855072463768116

0.7108433734939759

array([[130,  10],
       [ 24,  59]], dtype=int64)

0.8475336322869955

# h2O

In [41]:
# Load the H2O library and start up the H2O cluter locally on your machine
import h2o

import h2o

h2o.init(nthreads = -1)

h2o.remove_all

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.231-b11, mixed mode)
  Starting server from C:\Users\Krishna\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Krishna\AppData\Local\Temp\tmpkormyqt3
  JVM stdout: C:\Users\Krishna\AppData\Local\Temp\tmpkormyqt3\h2o_Krishna_started_from_python.out
  JVM stderr: C:\Users\Krishna\AppData\Local\Temp\tmpkormyqt3\h2o_Krishna_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Asia/Kolkata
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,23 days
H2O cluster name:,H2O_from_python_Krishna_9c3p00
H2O cluster total nodes:,1
H2O cluster free memory:,1.754 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


<function h2o.h2o.remove_all(retained=None)>

In [42]:
# Import H2O Grid Search:
from h2o.grid.grid_search import H2OGridSearch

# Import H2O GBM:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

In [43]:
train = h2o.H2OFrame(pd.concat([X_train,y_train],axis=1))
test = h2o.H2OFrame(pd.concat([X_test,y_test],axis=1))
train.head(2)
X_test.head(2)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Age,SibSp,Parch,Fare,Embarked=C,Embarked=Q,Embarked=S,Pclass=1,Pclass=2,Pclass=3,Sex=female,Sex=male,Surname=MASTER,Surname=MISS,Surname=MR,Surname=MRS,Surname=Others,Survived
24,1,0,26,0,0,1,0,1,0,1,0,0,0,0,1,0,1
21.774,0,0,7.8792,0,1,0,0,0,1,1,0,0,1,0,0,0,1


,Age,SibSp,Parch,Fare,Embarked=C,Embarked=Q,Embarked=S,Pclass=1,Pclass=2,Pclass=3,Sex=female,Sex=male,Surname=MASTER,Surname=MISS,Surname=MR,Surname=MRS,Surname=Others
205,2.0,0,1,10.4625,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
696,44.0,0,0,8.0500,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [44]:
train['Survived'] = train['Survived'].asfactor()
test['Survived'] = test['Survived'].asfactor()

In [45]:
# GBM hyperparameters
gbm_params1 = {'learn_rate': [0.01, 0.05,0.1], 
                'max_depth': [3, 5, 9],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.3, 0.5, 1.0]}

In [46]:
gbm_grid1 = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid1',
                          hyper_params=gbm_params1)
gbm_grid1.train(x=features, y=target,training_frame=train,ntrees = 100,seed = 69,validation_frame=test)


gbm Grid Build progress: |████████████████████████████████████████████████| 100%


In [47]:
test.shape

(223, 18)

In [48]:
gbm_gridperf1 = gbm_grid1.get_grid(sort_by='auc', decreasing=True)
print(gbm_gridperf1)

     col_sample_rate learn_rate max_depth sample_rate           model_ids  \
0                1.0       0.05         9         0.8  gbm_grid1_model_24   
1                0.5       0.01         9         1.0  gbm_grid1_model_47   
2                1.0       0.01         9         0.8  gbm_grid1_model_21   
3                1.0       0.05         3         1.0  gbm_grid1_model_33   
4                0.3       0.01         9         1.0  gbm_grid1_model_46   
5                0.5       0.01         9         0.8  gbm_grid1_model_20   
6                1.0       0.05         3         0.8   gbm_grid1_model_6   
7                0.3        0.1         3         1.0  gbm_grid1_model_34   
8                0.5       0.01         5         1.0  gbm_grid1_model_38   
9                0.3       0.01         9         0.8  gbm_grid1_model_19   
10               1.0       0.01         9         1.0  gbm_grid1_model_48   
11               1.0       0.01         5         1.0  gbm_grid1_model_39   

In [49]:
best_gbm_model = gbm_gridperf1.models[0]
gbm_perf = best_gbm_model.model_performance(test)
print(gbm_perf.auc())

0.9022805507745267


In [50]:
test_df1 = h2o.H2OFrame(test_df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [51]:
best_gbm_model.predict(test_df1).head(2)

gbm prediction progress: |████████████████████████████████████████████████| 100%


predict,p0,p1
0,0.951566,0.0484342
0,0.83717,0.16283


In [54]:
pred_test_bin = best_gbm_model.predict(test_df1)['predict']
pred_test_prob = best_gbm_model.predict(test_df1)['p1']
#logreg.score(test_df[features], test_df[target],)

gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


In [ ]:
h2o.exportFile(iris_hf, path = "/path/on/h2o/server/filesystem/iris.csv")

In [63]:
path

'C:/Users/Krishna/Documents/kaggle/titanic/input'

In [68]:
test_df2 = test_df1['PassengerId'].cbind(pred_test_bin).as_data_frame(use_pandas=True)

In [69]:
test_df2.to_csv('C:/Users/Krishna/Documents/kaggle/titanic/input/submit8h2o.csv',index = False)

In [70]:
5259/16554

0.31768756795940556

### for submitting

In [ ]:
pred_test_bin = XGB.predict(test_df[features])
pred_test_prob = XGB.predict_proba(test_df[features])[:,1]
#logreg.score(test_df[features], test_df[target],)

In [ ]:
pred_test_bin_sub = np.where(pred_test_prob > 0.5,1,0)#threshold[np.argmax(f_score)],1,0)
np.bincount(pred_test_bin_sub)
np.bincount(pred_test_bin)

In [ ]:
pd.concat([test_df[['PassengerId']],pd.DataFrame(pred_test_bin_sub,columns= ['Survived'],index = test_df.index)],axis = 1)\
.to_csv(f'{path}/titanic_submit7.csv', index=False)

In [ ]:
def xgb_feature_imp(model):
    """
    Gives the feature importance along with variable names for an XGB Model.
    """
    return(pd.DataFrame({'features' : model.get_booster().feature_names, 'importance' : model.feature_importances_}).\
           sort_values('importance',ascending = False))

feat_imp = xgb_feature_imp(model = XGB)
feat_imp

In [ ]:
def logreg_coef(model,data):    
    """
    Gives the intercepts and coefficients along with variable names for logistic regression.
    model : name of the logistic model
    data :  data on which logistic model was fit, column order must be same as that of the
            data on which model was fit
    """
    intercept = pd.DataFrame({'variable' : 'intercept', 'coefficient' : model.intercept_})
    coefficient = pd.DataFrame({'variable' : data.columns, 'coefficient' : model.coef_.transpose().flatten()})
    coefficient = coefficient.reindex(coefficient.coefficient.abs().sort_values(ascending = False).index)
    return(pd.concat([intercept,coefficient], axis = 0).reset_index(drop = True))

In [ ]:
lr_coefs = logreg_coef(model = logreg,data = train_df[features])
lr_coefs

In [ ]:
error aajaa

In [ ]:
precision, recall, threshold = precision_recall_curve(train_df[target],pred_prob)
precision.shape # last element is 1
recall.shape  # last element is 0
threshold.shape

In [ ]:
f_score = 2*precision*recall/(precision+recall)
f_score[-1]
np.argmax(f_score)
threshold[np.argmax(f_score)]

In [ ]:
pd.Series(threshold).describe()

In [ ]:
pr = precision[:-1]
rc = recall[:-1]
th  = threshold

th[np.where(pr==rc)]
pr[np.where(pr==rc)]
rc[np.where(pr==rc)]

In [ ]:
def plot_prec_recall_vs_tresh(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], 'b--', label='precision')
    plt.plot(thresholds, recalls[:-1], 'g--', label = 'recall')
    plt.xlabel('Threshold')
    plt.legend(loc='upper left')
    plt.ylim([0,1])

plot_prec_recall_vs_tresh(precision, recall, threshold)
plt.axvline(x = th[np.where(pr==rc)], ymin = 0, ymax= rc[np.where(pr==rc)],color = 'black')
plt.text(x = th[np.where(pr==rc)], y= rc[np.where(pr==rc)]/2, s = f'Th = {np.round(th[np.where(pr==rc)],2)}',\
         weight = 'heavy',color = 'red',  rotation = 'horizontal')
#plt.plot([th[np.where(pr==rc)],th[np.where(pr==rc)]],[0,rc[np.where(pr==rc)]], linestyle='--')
plt.plot(threshold, f_score[:-1], 'r--', label = 'F Score')
plt.show()

In [ ]:
x=[1,2,3]
y=[9,8,7]

plt.plot(x,y)
for a,b in zip(x, y): 
    plt.text(a, b, str(b))
plt.show()

In [ ]:
plt.axvline(x = th[np.where(pr==rc)], ymin = 0, ymax= rc[np.where(pr==rc)])

In [ ]:
precision[-40:-1]

In [ ]:
#https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-classification-in-python/
#https://towardsdatascience.com/beyond-accuracy-precision-and-recall-3da06bea9f6c
#https://www.programcreek.com/python/example/89259/sklearn.metrics.precision_recall_curve
_ = plt.plot(recall[:-1],precision[:-1], label='precision')
plt.plot([0, 1], [0.5, 0.5], linestyle='--')